# Data Science i molnet: "Azure ML SDK"-metoden

## Introduktion

I den här notebooken kommer vi att lära oss hur man använder Azure ML SDK för att träna, distribuera och använda en modell via Azure ML.

Förutsättningar:
1. Du har skapat en Azure ML-arbetsyta.
2. Du har laddat upp [Heart Failure-datasetet](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) till Azure ML.
3. Du har laddat upp den här notebooken till Azure ML Studio.

Nästa steg är:

1. Skapa ett experiment i en befintlig arbetsyta.
2. Skapa ett Compute-kluster.
3. Ladda datasetet.
4. Konfigurera AutoML med hjälp av AutoMLConfig.
5. Kör AutoML-experimentet.
6. Utforska resultaten och hitta den bästa modellen.
7. Registrera den bästa modellen.
8. Distribuera den bästa modellen.
9. Använd slutpunkten.

## Azure Machine Learning SDK-specifika importer


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Initiera arbetsyta  
Initiera ett arbetsyteobjekt från sparad konfiguration. Se till att konfigurationsfilen finns på plats i .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Skapa ett Azure ML-experiment

Låt oss skapa ett experiment med namnet 'aml-experiment' i det arbetsutrymme vi just initierade.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Skapa ett beräkningskluster
Du behöver skapa ett [beräkningsmål](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) för din AutoML-körning.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data
Se till att du har laddat upp datasetet till Azure ML och att nyckeln har samma namn som datasetet.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML-konfiguration


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML-körning


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Distribuera den bästa modellen

Kör följande kod för att distribuera den bästa modellen. Du kan se distributionsstatusen i Azure ML-portalen. Detta steg kan ta några minuter.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Använd slutpunkten
Du kan lägga till indata i följande exempel på indata.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
